In [30]:
from preprocess_income_data import preprocess_data

X_train, X_test, X_val, y_train, y_val, y_test = preprocess_data()

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.linear_model import LogisticRegression
import numpy as np
from const import N_SPLITS, RANDOM_STATE, LOG_REG, LOG_REG_ID

ALGORITHM_NAME = LOG_REG
ALGORITHM_ID = LOG_REG_ID

kf = KFold(n_splits=20, random_state=RANDOM_STATE, shuffle=True)
C = [0.001, 0.01, 0.1, 1, 10, 100, 1000]

solver = ['lbfgs', 'liblinear']

steps = [(ALGORITHM_NAME, LogisticRegression(
    random_state=RANDOM_STATE,
    max_iter=1000
    ))]


pipeline = Pipeline(steps)

parameters = {
    f"{ALGORITHM_NAME}__C": C,
    f"{ALGORITHM_NAME}__solver": solver,
}

cv = GridSearchCV(pipeline, param_grid=parameters, cv=kf, scoring="accuracy")

cv.fit(X_train, y_train)


GridSearchCV(cv=KFold(n_splits=20, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('logistic_regression',
                                        LogisticRegression(max_iter=1000,
                                                           random_state=42))]),
             param_grid={'logistic_regression__C': [0.001, 0.01, 0.1, 1, 10,
                                                    100, 1000],
                         'logistic_regression__solver': ['lbfgs', 'liblinear']},
             scoring='accuracy')

In [32]:
cv.best_score_

0.8524920890252382

In [33]:
from process_cv_results import process_cv_scores, process_best_params, get_classification_metrics, results_to_csv
accuracy_scores_df = process_cv_scores(cv, algorithm_id=ALGORITHM_ID)
accuracy_scores_df.head()


,algorithm_id,accuracy_score
0,2,0.836096
1,2,0.832413
2,2,0.852056
3,2,0.853898
4,2,0.856967


In [34]:
best_params_df = process_best_params(
    cv, algorithm_id=ALGORITHM_ID, algorithm_name=ALGORITHM_NAME)
best_params_df.head()


,algorithm_id,hyperparameter_name,best_value
0,2,C,10
1,2,solver,lbfgs


In [35]:
y_pred = cv.predict(X_test)
classification_metrics_df = get_classification_metrics(
    y_test, y_pred, cv, ALGORITHM_ID, ALGORITHM_NAME)

classification_metrics_df.head()


[[5824  420]
 [ 770 1126]]

---


,algorithm_id,algorithm_name,best_accuracy,precision,recall,roc_auc_score,true_positives,false_positives,false_negatives,true_negatives
0,2,logistic_regression,0.852492,0.728331,0.593882,0.763309,5824,420,770,1126


In [36]:
dfs = [accuracy_scores_df, best_params_df, classification_metrics_df]

results_to_csv(dfs, ALGORITHM_ID)
